In [3]:
#!/usr/bin/env python3
"""
Script for predicting bounding boxes for the RSNA pneumonia detection challenge
by Phillip Cheng, MD MS
"""

import os
import sys
import time
import json
import numpy as np
import pandas as pd

# import utility functions
import util
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
# This is a modified version of keras-retinanet 0.4.1
# which includes a score metric to estimate the RSNA score
# at the threshold giving the maximum Youden index.
sys.path.append("keras-retinanet")
from keras_retinanet import models


with open('settings.json') as json_data_file:
    json_data = json.load(json_data_file)
# model_path = json_data["MODEL1_50"]
model_path = "models/resnet101_800_4_0.229.h5"
model = models.load_model(model_path, backbone_name='resnet101', convert=True, nms=False)


test_jpg_dir = json_data["TEST_JPG_DIR"]
submission_dir = json_data["SUBMISSION_DIR"]

sz = 800

# threshold for non-max-suppresion for each model
nms_threshold = 0

# threshold for including boxes from model 1
#score_threshold = 0.04
score_threshold = 0.04
# threshold for including isolated boxes from either model
solo_min = 0.15

test_ids = []
test_outputs = []

start = time.time()

for i, fname in enumerate(os.listdir(test_jpg_dir)):
    print(f"Predicting boxes for image # {i+1}\r", end="")
    fpath = os.path.join(test_jpg_dir, fname)
    fid = fname[:-4]

    boxes_pred, scores = util.get_detection_from_file(fpath, model, sz)

    indices = np.where(scores > score_threshold)[0]
    scores = scores[indices]
    boxes_pred = boxes_pred[indices]
    boxes_pred, scores = util.nms(boxes_pred, scores, nms_threshold)
    output = ''
    for j, bb in enumerate(boxes_pred):
        x1 = int(bb[0])
        y1 = int(bb[1])
        w = int(bb[2]-x1+1)
        h = int(bb[3]-y1+1)
        output += f'{scores[j]:.3f} {x1} {y1} {w} {h} '
    test_ids.append(fid)
    test_outputs.append(output)
print()
end = time.time()
# print execution time
print(f"Elapsed time = {end-start:.3f} seconds")

test_df = pd.DataFrame({'patientId': test_ids, 'PredictionString': test_outputs},
                       columns=['patientId', 'PredictionString'])
if not os.path.exists(submission_dir):
    os.mkdir(submission_dir)
test_df.to_csv(os.path.join(submission_dir, "resnet101_800_4_0.229_0.04.csv"), index = False)

Predicting boxes for image # 3000
Elapsed time = 263.537 seconds
